In [1]:
import pandas as pd
import os
from pathlib import Path
import cv2
import numpy as np
from matplotlib import pyplot as plt
import matplotlib as mpl
from sklearn.model_selection import train_test_split, KFold
from PIL import Image
from ImageAnalysis import ImageAnalysis
from ImageTableExtraction import ImageTableExtraction
from SudokuDetection import SudokuDetection
from solvers.sudoku.sudoku_solver import SudokuSolver

In [2]:
mpl.rcParams['figure.dpi']= 200
plt.rcParams['figure.figsize'] = [5,4]

In [3]:
data_folder = "../data/original/"
annotated_folder = "../data/training_data/"
testdata_folder = "../data/testdata_dont_touch/"

In [4]:
def read_samples(folder):
    samples = []
    for file in os.listdir(data_folder):
         filename = os.fsdecode(file)
         if filename.endswith(".jpg"):
            fullname = data_folder + file
            fullname_no_extension = data_folder + Path(fullname).stem
            outcome = pd.read_json(fullname_no_extension + ".json")
            samples.append((filename, outcome))
    return samples

In [5]:
samples = read_samples(data_folder)

In [6]:
def eval(samples):
    success = [x for x in samples if len(x[1]["solution"]) > 0]
    print(len(success))
    print(len(samples))
    return len(success) / len(samples)

In [7]:
eval(samples)

63
217


0.2903225806451613

In [8]:
X_train, X_test, y_train, y_test = train_test_split(samples, samples, test_size=0.20, random_state=42)
print(len(X_train))
print(len(X_test))

173
44


In [9]:
def move_annotated_files(samples, source_folder, destination_folder):
    for test_sample in X_test:
        image_filepath = test_sample[0]
        image_fullpath = source_folder + image_filepath
        image_target = destination_folder + image_filepath
        name_no_ext = Path(image_filepath).stem
        annotation_filepath = name_no_ext + ".xml"
        annotation_fullpath = source_folder + annotation_filepath
        annotation_target = destination_folder + annotation_filepath
        filename = os.fsdecode(image_filepath)
        os.rename(image_fullpath, image_target)
        os.rename(annotation_fullpath, annotation_target)

In [157]:
class SudolverModel():
    def __init__(self):
        self.extractor = None
        self.persp_transform = False
        self.filters = []
        self.solver = SudokuSolver()
        self.detector = SudokuDetection(ImageAnalysis(), ImageTableExtraction())
        
    def predict(self, image_rgb):
        try:
            result = image_rgb
            if self.extractor is not None:
                preprocessed = self.extractor.preprocess(result)
                result = self.extractor.predict(preprocessed, self.persp_transform)
                if result is None:
                    print("YOLO detected not sudoku")
                    return 0
            for f in self.filters:
                result = f.apply_filter(result)
            return self.scan_and_solve(result)
        except Exception as ex:
            print(ex)
            return 0
    
    def get_params(self):
        desc = ""
        if self.extractor is not None:
            desc += "Extractor.Activated,"
        if self.persp_transform:
            desc += "PerspTransform.Activated"
        for f in self.filters:
            desc += f.get_params() + ","
        return desc
    
    def enable_extractor(self):
        self.extractor = SudokuExtractor()
        
    def use_perspective_transform(self):
        self.persp_transform = True
    
    def add_filter(self, new_filter):
        self.filters.append(new_filter)
    
    def scan_and_solve(self, image_rgb):
        result = None
        try:
            success, encoded_image = cv2.imencode('.jpg', image_rgb)
            content = encoded_image.tobytes()
            success, result = self.detector.detect(content)
            if success:
                solution = self.solver.solve(result)
                return 1
            else:
                print(result)
                return 0
        except Exception as ex:
            print(ex)
            if result is not None:
                print(result)
            return 0

In [178]:
from tqdm import tqdm
def eval_all(samples):
    extractor = SudokuExtractor()
    sharp_variants = (UnsharpMasking(10), Laplacian(10), UnsharpMasking(5), Laplacian(5), None)
    results = []
    total = len(samples)
    for sharp in sharp_variants:
            sudolver = SudolverModel()
            sudolver.enable_extractor()
            if sharp is not None:
                sudolver.add_filter(sharp)
            model_total = 0
            for sample in tqdm(samples):
                filename = sample[0]
                image = cv2.imread(data_folder + filename)
                image_rgb = to_rgb_from_bgr(image)
                success = sudolver.predict(image_rgb)
                model_total += success
            model_desc = sudolver.get_params()
            eval_result = (model_desc, model_total / total)
            print(eval_result)
            results.append(eval_result)
    return results

In [179]:
def read_image(filepath):
    return cv2.imread(filepath)

In [180]:
def to_rgb_from_bgr(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

In [181]:
def to_bgr_from_rgb(image):
    return cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

In [182]:
def to_grayscale_from_bgr(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

In [183]:
def eq_hist_from_bgr(image):
    return cv2.equalizeHist(to_grayscale_from_bgr(image))

In [184]:
def unsharp_masking(image, k=10):
    normalized = image / 255.0
    blurred = cv2.GaussianBlur(normalized, [5, 5], 3, 3)
    mask = normalized - blurred
    sharp = normalized + k*mask
    sharp[sharp > 1] = 1
    sharp[sharp < 0] = 0
    sharp = sharp * 255
    sharp = sharp.astype(np.uint8)
    return sharp

In [185]:
def laplacian(image, alpha=10):
    normalized = image / 255.0
    laplacian = cv2.Laplacian(normalized, cv2.CV_64F)
    sharp = normalized - alpha * laplacian
    sharp[sharp > 1] = 1
    sharp[sharp < 0] = 0
    sharp = sharp * 255
    sharp = sharp.astype(np.uint8)
    return sharp

In [186]:
def save_higher_dpi(source_filepath, target_filepath, new_dpi=150):
    im = Image.open(source_filepath)
    im.save(target_filepath, dpi=(new_dpi,new_dpi))

In [187]:
def perspective_transform(image, conjugates, output_size=512):
    output = np.float32([[0, 0], [output_size - 1, 0], [output_size - 1, output_size - 1], [0, output_size - 1]])
    matrix = cv2.getPerspectiveTransform(conjugates, output)
    return cv2.warpPerspective(image, matrix, (output_size, output_size), cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT, borderValue=(0, 0, 0))

In [188]:
filepath = annotated_folder + X_train[34][0]

In [189]:
image = read_image(filepath)
#plt.imshow(to_rgb_from_bgr(image))

In [190]:
#plt.imshow(perspective_transform(image, np.float32([[75,150], [418,59], [442,443], [29,438]])))

In [191]:
#plt.imshow(to_grayscale_from_bgr(image), cmap='gray')

In [192]:
#plt.imshow(eq_hist_from_bgr(image), cmap='gray')

In [193]:
sharp = unsharp_masking(image, k=15)

In [194]:
sharp = laplacian(image, alpha=10)

In [195]:
from imgaug.augmenters.meta import Sequential
import imgaug.augmenters as iaa
import imageio
import torch

class SudokuExtractor():
    def __init__(self):
        self.count = 0
        self.model = torch.hub.load('C:/Projects/sudolver-models/data/model/yolov5', 'custom', path='C:/Projects/sudolver-models/data/trained_model/best_2022-04-09.pt', source='local') 

    def preprocess(self, image):
        aug = [iaa.CenterPadToSquare(), iaa.Resize({"height": 640, "width": 640})]
        seq = iaa.Sequential(aug)
        return seq(image=image)
        
    def predict(self, image, persp_transform):
        result = self.model(image)
        if len(result) > 0:
            pred = result.pandas()
            xmin, ymin, xmax, ymax = (int(pred.xyxy[0]['xmin'].values[0]), int(pred.xyxy[0]['ymin'].values[0]), int(pred.xyxy[0]['xmax'].values[0]), int(pred.xyxy[0]['ymax'].values[0]))
            if persp_transform:
                return to_rgb_from_bgr(perspective_transform(image, self.pred_to_conjugates(xmin, ymin, xmax, ymax)))
            else:
                img = image[ymin:ymax+1, xmin:xmax+1]
                cv2.imwrite("C:\Projects\sudolver-models\data\detected
        else:
            return None
        
    def pred_to_conjugates(self, xmin, ymin, xmax, ymax):
        return np.float32([[xmin, ymin], [xmax, ymin], [xmax, ymax], [xmin, ymax]])

In [196]:
class ImageFilter():
    def apply_filter(self, image):
        pass
    
    def get_params(self):
        pass
    
class UnsharpMasking(ImageFilter):
    def __init__(self, k=5):
        self.k = k
        
    def apply_filter(self, image_rgb):
        return unsharp_masking(image_rgb, self.k)
    
    def get_params(self):
        return f"UnsharpMasking (k={self.k})"
    
class Laplacian(ImageFilter):
    def __init__(self, alpha=5):
        self.alpha = alpha
        
    def apply_filter(self, image_rgb):
        return laplacian(image_rgb, self.alpha)
    
    def get_params(self):
        return f"Laplacian (alpha={self.alpha})"
    
class HigherDPI(ImageFilter):
    def __init__(self, dpi=150):
        self.dpi = dpi
        
    def apply_filter(self, image_rgb):
        im = Image.fromarray(image_rgb)
        temp_filename = "C:/Projects/sudolver-models/data/high_dpi/temp.jpg"
        im.save(temp_filename, dpi=(self.dpi, self.dpi))
        return to_rgb_from_bgr(read_image(temp_filename))
    
    def get_params(self):
        return f"Higher DPI = {self.dpi}"

In [ ]:
results = eval_all(X_train)
print(results)